To change the optimization function to minimize utility costs based on hourly tariffs and demand charges, we need to adjust both the objective function and constraints. Here's the detailed process:
Explanation

    Objective Function:
        We need to modify the objective function to minimize the sum of the hourly utility costs and the demand charges.
        Utility Cost: This is calculated as the product of the hourly load profile and the hourly utility tariff.
        Demand Charge: This is the product of the peak load and the demand charge rate.

    Variables:
        The existing variables (chargeprofiles, dischargeprofiles, total_load, max_load) are sufficient, but we'll also need an array for hourly utility tariffs.

    Constraints:
        The existing constraints related to the load profiles and state of charge will remain the same.

Code Changes

    Add Utility Tariffs and Demand Charge Rate:
        Define an array for hourly utility tariffs.
        Define a variable for the demand charge rate.

    Modify the Objective Function:
        Incorporate the utility cost and demand charge into the objective function.

Here are the changes to the code:
Step-by-Step Code Changes

    Add Utility Tariffs and Demand Charge Rate:
        Define hourly_tariffs as an array with 24 elements representing the tariff for each hour.
        Define demand_charge_rate as the cost per unit of peak load.

In [1]:
hourly_tariffs = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]  # Example tariffs
demand_charge_rate = 10  # Example demand charge rate


In [2]:
len(hourly_tariffs)

24

Modify the Objective Function:

    Add the utility cost and demand charge components to the objective function.

In [3]:
def V2G_optimization():
    # initiate the problem statement
    model = lp.LpProblem('Minimize_Utility_Cost', lp.LpMinimize)

    # define optimization variables
    veh_V2G = range(n_V2G_Veh)
    time_Interval = range(24)
    chargeprofiles = lp.LpVariable.dicts('charging_profiles', ((i, j) for i in veh_V2G for j in time_Interval), lowBound=0, upBound=power_managed_Uppper)
    chargestates = lp.LpVariable.dicts('charging_states', ((i, j) for i in veh_V2G for j in time_Interval), cat='Binary')
    dischargeprofiles = lp.LpVariable.dicts('discharging_profiles', ((i, j) for i in veh_V2G for j in time_Interval), lowBound=power_V2G_Lower, upBound=0)
    dischargestates = lp.LpVariable.dicts('discharging_states', ((i, j) for i in veh_V2G for j in time_Interval), cat='Binary')
    total_load = lp.LpVariable.dicts('total_load', time_Interval, lowBound=0)
    max_load = lp.LpVariable('max_load', lowBound=0)
    min_load = lp.LpVariable('min_load', lowBound=0)

    # define objective function
    utility_cost = lp.lpSum([total_load[t] * hourly_tariffs[t] for t in time_Interval])
    demand_charge = max_load * demand_charge_rate
    model += utility_cost + demand_charge

    # define constraints
    for t in time_Interval:  # constraint 1 & 2: to identify the max and min loads
        model += total_load[t] <= max_load
        # model += total_load[t] >= min_load

    for t in time_Interval:  # constraint 3: calculate the total load at each time interval t
        model += lp.lpSum([chargeprofiles[i, t] for i in veh_V2G]) + lp.lpSum([dischargeprofiles[i, t] * discharge_efficiency for i in veh_V2G]) + base_Load[t] + unmanaged_Load[t] == total_load[t]

    for i in veh_V2G:  # constraint 4: constraint on charging powers for each EV: only optimize the charge profile between start and end charging time
        temp_start = v2g_startingTime[i]
        temp_end = v2g_endingTime[i]
        if temp_start >= temp_end:
            for t in range(temp_end):
                model += chargestates[i, t] + dischargestates[i, t] <= 1
                model += chargeprofiles[i, t] <= chargestates[i, t] * power_managed_Uppper
                model += chargeprofiles[i, t] >= chargestates[i, t] * power_managed_Lower
                model += dischargeprofiles[i, t] <= dischargestates[i, t] * power_V2G_Upper
                model += dischargeprofiles[i, t] >= dischargestates[i, t] * power_V2G_Lower
            for t in range(temp_end, temp_start, 1):
                model += chargeprofiles[i, t] == 0
                model += chargestates[i, t] == 0
                model += dischargeprofiles[i, t] == 0
                model += dischargestates[i, t] == 0
            for t in range(temp_start, 24, 1):
                model += chargestates[i, t] + dischargestates[i, t] <= 1
                model += chargeprofiles[i, t] <= chargestates[i, t] * power_managed_Uppper
                model += chargeprofiles[i, t] >= chargestates[i, t] * power_managed_Lower
                model += dischargeprofiles[i, t] <= dischargestates[i, t] * power_V2G_Upper
                model += dischargeprofiles[i, t] >= dischargestates[i, t] * power_V2G_Lower

        if temp_start < temp_end:
            for t in range(temp_start):
                model += chargeprofiles[i, t] == 0
                model += chargestates[i, t] == 0
                model += dischargeprofiles[i, t] == 0
                model += dischargestates[i, t] == 0
            for t in range(temp_start, temp_end, 1):
                model += chargestates[i, t] + dischargestates[i, t] <= 1
                model += chargeprofiles[i, t] <= chargestates[i, t] * power_managed_Uppper
                model += chargeprofiles[i, t] >= chargestates[i, t] * power_managed_Lower
                model += dischargeprofiles[i, t] <= dischargestates[i, t] * power_V2G_Upper
                model += dischargeprofiles[i, t] >= dischargestates[i, t] * power_V2G_Lower
            for t in range(temp_end, 24, 1):
                model += chargeprofiles[i, t] == 0
                model += chargestates[i, t] == 0
                model += dischargeprofiles[i, t] == 0
                model += dischargestates[i, t] == 0

    for i in veh_V2G:  # constraint 5: SOC constraint, cannot be greater than 1, end_SOC must be above certain levels
        temp_start = v2g_startingTime[i]
        temp_end = v2g_endingTime[i]
        temp_startSOC = v2g_startingSOC[i]
        if temp_start >= temp_end:
            for t in range(temp_start + 1, 24, 1):
                temp_timer = range(temp_start, t, 1)
                model += temp_startSOC + lp.lpSum([chargeprofiles[i, tn] * charge_efficiency / batteryCapacity for tn in temp_timer]) + lp.lpSum([dischargeprofiles[i, tn] * (1 / batteryCapacity) for tn in temp_timer]) <= 1
            for t in range(0, temp_end + 1, 1):
                temp_timer = range(0, t, 1)
                model += temp_startSOC + lp.lpSum([chargeprofiles[i, tn] * charge_efficiency / batteryCapacity for tn in range(temp_start, 24, 1)]) + lp.lpSum([chargeprofiles[i, tn] * charge_efficiency / batteryCapacity for tn in temp_timer]) + lp.lpSum([dischargeprofiles[i, tn] * (1 / batteryCapacity) for tn in range(temp_start, 24, 1)]) + lp.lpSum([dischargeprofiles[i, tn] * (1 / batteryCapacity) for tn in temp_timer]) <= 1
            if end_SOC == 2:
                model += temp_startSOC + lp.lpSum([chargeprofiles[i, tn] * charge_efficiency / batteryCapacity for tn in range(temp_start, 24, 1)]) + lp.lpSum([chargeprofiles[i, tn] * charge_efficiency / batteryCapacity for tn in temp_timer]) + lp.lpSum([dischargeprofiles[i, tn] * (1 / batteryCapacity) for tn in range(temp_start, 24, 1)]) + lp.lpSum([dischargeprofiles[i, tn] * (1 / batteryCapacity) for tn in temp_timer]) == 1

        if temp_start < temp_end:
            for t in range(temp_start + 1, temp_end + 1, 1):
                temp_timer = range(temp_start, t, 1)
                model += temp_startSOC + lp.lpSum([chargeprofiles[i, tn] * charge_efficiency / batteryCapacity for tn in temp_timer]) + lp.lpSum([dischargeprofiles[i, tn] * (1 / batteryCapacity) for tn in temp_timer]) <= 1
            if end_SOC == 2:
                model += temp_startSOC + lp.lpSum([chargeprofiles[i, tn] * charge_efficiency / batteryCapacity for tn in temp_timer]) + lp.lpSum([dischargeprofiles[i, tn] * (1 / batteryCapacity) for tn in temp_timer]) == 1

    # solve the model
    status = model.solve()
    print(lp.LpStatus[status])
    print(lp.value(max_load))

    return chargeprofiles, dischargeprofiles, total_load


Summary of Changes

    Added New Variables:
        hourly_tariffs: Array representing the hourly utility tariffs.
        demand_charge_rate: The cost per unit of peak load.

    Modified the Objective Function:
        The objective function now includes the sum of hourly utility costs and the demand charge.

    Kept the Constraints the Same:
        Existing constraints on max and min loads, total load calculation, charging and discharging profiles, and SOC constraints are retained.